In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import sys
sys.path.append("../")
import fast_scBatch
from tqdm import trange

In [ ]:
for i in trange(1, 30):
	cell = anndata.read(f"../../../simulation/data/data{i}.h5ad")
	batch = cell.obs[["Batch"]].copy()
	cells = cell.to_df().T
	# corr = pd.read_csv(f"../../quantnorm/results/quantnorm_simu{i}.csv", index_col=0)
	corr = pd.read_csv(f"../../fastqn/corr_simu.csv", index_col=0)
	corr.columns = cells.columns
	corr.index = cells.columns
	p, n = cells.shape
	res = fast_scBatch.solver(cells, corr, batch, p=0.3, k=20,
		lr=(0.0002, 0.0001, 0.0001), EPOCHS=(0, 0, 2000), verbose=False)
	adata = anndata.AnnData(X=res.T, obs=cell.obs, var=cell.var)
	adata.write(f"../results/fastscbatch_simu{i}.h5ad")

In [ ]:
print(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.pl.umap(adata, color=["Batch", "Group"], wspace=0.5, frameon=False)